# Subset ERA5 data needed to calculate atmospheric rivers

This program:

Reads in U, V, Q at 14 levels from ERA5 for Nov-Dec-Jan-Feb-Mar 1986/87 to 2015/16

Data are located in: 

    /glade/collections/rda/data/ds633.0/e5.oper.an.pl/yyyymm/e5.oper.an.pl.128_133_variable.ll025sc.yyyymmddhh_yyyymmddhh.grb

Subsets to the PNA region
    
    Defined as: 150-300E; 20-80N
    
Choose whether to interpolate the data to a N32 (126x64) Gaussian Grid or an N128 (512 x 256) grid, or select the native grid.

Writes datasets out to a netcdf files containing a single variable

Output file is located in: `/glade/work/mkorendy/metisregimes/era5.AR.variable_pna_NDJFM.1986-2016.nc`

## Constants

In [1]:
# Choose interpolation: N32 or N128 or native
interpolate = 'N128'

## Import

In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob

## Function to get the PNA region for the variable at the 14 levels, and interpolate to new grid

In [3]:
def preproc(ds):
    
    pressures = [1000, 925, 850, 700, 600, 500, 400, 300, 250, 200, 100, 50, 30, 10]

    # Extract PNA region for every 12 hours for the 14 pressure levels
    ds=ds.sel(time=ds['time'][0::12],isobaricInhPa = pressures,latitude=slice(80,20),longitude=slice(150,300))
    
    #Extract PNA region
    #ds = ds.sel(latitude=slice(20,80),longitude=slice(150,300))

    # Create a new xarray dataset for the variable
    #ds_tmp=xr.DataArray(ds,
    #                    coords={'time':ds['time'].values, 
    #                            'levs':ds['isobaricInhPa'].values,
    #                            'lat':ds['latitude'].values,
    #                            'lon': ds['longitude'].values},
    #                        dims=['time','levs','latitude','longitude'])        
    #ds_tmp=ds_tmp.to_dataset(name=variable)
    
    ds = ds.rename(latitude='lat',longitude='lon')
    
    # Interpolate data to coarser grid
    if interpolate == 'N32':
        ds_new_grid=xr.open_dataset('tempgaussian.128x64.nc')
        ds_new_grid=ds_new_grid.rename({'temp':variable}).sel(lat=slice(80,20),lon=slice(150,300))
        #ds=ds_tmp.interp_like(ds_new_grid)
        ds=ds.interp_like(ds_new_grid)
    elif interpolate == 'N128':
        ds_new_grid=xr.open_dataset('/glade/work/mkorendy/metisregimes/metis.vqvi_pna_NDJFM.1986-2016.N128.nc')
        lat = ds_new_grid.sel(lat=slice(20,80),lon=slice(150,300)).lat
        lon = ds_new_grid.sel(lat=slice(20,80),lon=slice(150,300)).lon
        go = np.empty([len(lat),len(lon)])
        ds_new_grid = xr.DataArray(go,coords={'lat':lat,'lon':lon}, dims=['lat','lon']).to_dataset(name=variable)
        #ds=ds_tmp.interp_like(ds_new_grid)
        ds=ds.interp_like(ds_new_grid)
    else:
        print('native')
        
    return ds

## Define months and years to get


In [4]:
# Months list (Nov, Dec, Jan, Feb, Mar)
mnums=['11', '12', '01', '02', '03']
#mnums = ['04','05','06','07','08','09']
#mnums = ['04']
mnums = ['03']

# Dates
sdate='19861101'
edate='20160331'

# Years List
yrs_list=np.arange(1986,2016)
#yrs_list = np.arange(1987,1988)
yrs_list = np.arange(2015,2016)
print(yrs_list)

#Variable
variable = 'q'

#Names for key-wargs
if variable == 'u':
    name = 'U component of wind'
    file = '/*u.*uv.*grb'
elif variable == 'v':
    name = 'V component of wind'
    file = '/*v.*uv.*grb'
elif variable == 'q':
    name = 'Specific humidity'
    file = '/*q*grb'
    #file = '/*q*nc'

[2015]


## Define file path and names¶

In [5]:
# Input
path='/glade/collections/rda/data/ds633.0/e5.oper.an.pl/'

# Output
out_path='/glade/work/mkorendy/metisregimes/'

if interpolate == 'N32':
    ofname = 'era5.AR.' + variable + '_pna_NDJFM.1986-2016.nc'
elif interpolate == 'native':
    ofname = 'era5.AR.' + variable + '_pna_NDJFM.1986-2016.reg_grid.nc'
else:
    ofname = 'era5.AR.' + variable + '_pna_NDJFM.1986-2016.N128.nc'
#ofname='era5.AR.' + variable + '_pna_AMJJAS.1986-2016.nc'
#ofname = 'era5.AR.' + variable + '_pna_O.1986-2016.nc'
outfile=out_path+ofname

In [6]:
print(outfile)

/glade/work/mkorendy/metisregimes/era5.AR.q_pna_NDJFM.1986-2016.N128.nc


In [7]:
#test = xr.open_dataset('/glade/collections/rda/data/ds633.0/e5.oper.an.pl/199710/e5.oper.an.pl.128_133_q.ll025sc.1997102500_1997102523.nc')

In [8]:
pressures = [1000, 925, 850, 700, 600, 500, 400, 300, 250, 200, 100, 50, 30, 10]
#test.sel(time=test['time'][0::12], level = pressures,latitude=slice(80,20),longitude=slice(150,300))

## Read in the files for all months and years, and prepocess to subset data for U, V, or Q in the PNA region & interpolate to coarser grid

In [9]:
ds_months = []

for mnum in mnums:

    fnames = [f'{path}{year}{mnum}' + file for year in yrs_list]

    filesVar=[]
    for files in fnames:
        f2=glob.glob(files)
        for f in f2:
            filesVar.append(f)
    
    filesVar=sorted(filesVar)
    
    ds = xr.open_mfdataset(filesVar,engine='cfgrib',
                           combine='nested',concat_dim=['time'],
                           backend_kwargs={'indexpath':'',
                                           'filter_by_keys':{'name': name}},
                          preprocess=preproc)
    
    ds_months.append(ds)

ds_months = xr.combine_nested(ds_months, concat_dim='time')

ValueError: unrecognized engine cfgrib must be one of: ['netcdf4', 'scipy', 'pydap', 'store', 'zarr']

In [ ]:
ds_months

## Write to netcdf

In [87]:
ds_months.to_netcdf(outfile)

In [88]:
print(outfile)

/glade/work/mkorendy/metisregimes/era5.AR.q_pna_NDJFM.1986-2016.N128.nc
